<a href="https://colab.research.google.com/github/rbutti/machine-learning-cohort/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install linkedin-jobs-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.5.1
    Uninstalling websocket-client-1.5.1:
      Successfully uninstalled websocket-client-1.5.1


In [6]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.arXnMA8Q4X/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Z46lkjjp7S/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.SICoNii7nc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [7]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://p

<ipython-input-7-d40b5adfff85>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [10]:
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, \
    OnSiteOrRemoteFilters

# Change root logger level (default is WARN)
logging.basicConfig(level=logging.INFO)


# Fired once for each successfully processed job
def on_data(data: EventData):
    print('[ON_DATA]', data.title, data.company, data.company_link, data.date, data.link, data.insights,
          len(data.description))


# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
    print('[ON_METRICS]', str(metrics))


def on_error(error):
    print('[ON_ERROR]', error)


def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None,  # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver) 
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=40  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)

queries = [
    # Query(
    #     options=QueryOptions(
    #         limit=27  # Limit the number of jobs to scrape.            
    #     )
    # ),
    Query(
        query='Engineer',
        options=QueryOptions(
            locations=['United States', 'Europe'],
            apply_link=True,  # Try to extract apply link (easy applies are skipped). If set to True, scraping is slower because an additional page mus be navigated. Default to False.
            skip_promoted_jobs=True,  # Skip promoted jobs. Default to False.
            page_offset=2,  # How many pages to skip
            limit=5,
            filters=QueryFilters(
                company_jobs_url='https://www.linkedin.com/jobs/search/?f_C=1068%2C163001%2C1067&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3613608940%2C3613614226%2C3612306061%2C3600564871%2C3577748932%2C3604566411%2C3601478332%2C3604565437%2C3607203148&lipi=urn%3Ali%3Apage%3Acompanies_company_jobs_jobs%3B47825738-b709-4b86-bd08-5d39b560d8c8',  # Filter by companies.                
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
                on_site_or_remote=[OnSiteOrRemoteFilters.REMOTE],
                experience=[ExperienceLevelFilters.MID_SENIOR]
            )
        )
    ),
]

scraper.run(queries)

INFO:li:scraper:('Using strategy AnonymousStrategy',)
INFO:li:scraper:('Starting new query', "Query(query=Engineer options=QueryOptions(limit=5 locations=['United States', 'Europe'] filters=QueryFilters(company_jobs_url=https://www.linkedin.com/jobs/search/?f_C=1068%2C163001%2C1067&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3613608940%2C3613614226%2C3612306061%2C3600564871%2C3577748932%2C3604566411%2C3601478332%2C3604565437%2C3607203148&lipi=urn%3Ali%3Apage%3Acompanies_company_jobs_jobs%3B47825738-b709-4b86-bd08-5d39b560d8c8 relevance=RelevanceFilters.RECENT time=TimeFilters.MONTH type=[<TypeFilters.FULL_TIME: 'F'>, <TypeFilters.INTERNSHIP: 'I'>] experience=[<ExperienceLevelFilters.MID_SENIOR: '4'>] on_site_or_remote=[<OnSiteOrRemoteFilters.REMOTE: '2'>]) optimize=False apply_link=True skip_promoted_jobs=True page_offset=2))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:42363')
INFO:li:scraper:('[Engineer][United States]', 'Openin

[ON_DATA] Senior Lead Software Engineer - SaaS/Cloud JPMorgan Chase & Co.  2023-05-02 https://www.linkedin.com/jobs/view/senior-lead-software-engineer-saas-cloud-at-jpmorgan-chase-co-3590161991?refId=QR2Hj%2FhI7NntBLOATqmv3Q%3D%3D&trackingId=DWtofBgNiu3vM7BPx0MMTQ%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card [] 2442


INFO:li:scraper:('[Engineer][United States][2]', 'Processed')


[ON_DATA] Lead Software Engineer - Full Stack AWS JPMorgan Chase & Co.  2023-05-18 https://www.linkedin.com/jobs/view/lead-software-engineer-full-stack-aws-at-jpmorgan-chase-co-3606153331?refId=QR2Hj%2FhI7NntBLOATqmv3Q%3D%3D&trackingId=KMPoN029sBcWsP%2B7dz6YKw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card [] 1593


INFO:li:scraper:('[Engineer][United States][3]', 'Processed')


...

[ON_DATA] Healthcare Payments Solution Engineer - Vice President JPMorgan Chase & Co.  2023-05-21 https://www.linkedin.com/jobs/view/healthcare-payments-solution-engineer-vice-president-at-jpmorgan-chase-co-3607288358?refId=QR2Hj%2FhI7NntBLOATqmv3Q%3D%3D&trackingId=5e0DelfXRk5fkgDFiaONRg%3D%3D&position=6&pageNum=0&trk=public_jobs_jserp-result_search-card [] 5926


INFO:li:scraper:('[Engineer][United States][5]', 'Processed')


[ON_DATA] Global Real Estate Engineering & Operations - Resiliency, VP JPMorgan Chase & Co.  2023-05-20 https://www.linkedin.com/jobs/view/global-real-estate-engineering-operations-resiliency-vp-at-jpmorgan-chase-co-3588045571?refId=QR2Hj%2FhI7NntBLOATqmv3Q%3D%3D&trackingId=ZGPZEkaSyFDb%2BTUD%2FSZsog%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card [] 6618


INFO:li:scraper:('Chrome debugger url', 'http://localhost:42915')
INFO:li:scraper:('[Engineer][Europe]', 'Opening https://www.linkedin.com/jobs/search?keywords=Engineer&location=Europe&f_C=1068%2C163001%2C1067&sortBy=DD&f_TPR=r2592000&f_JT=F%2CI&f_E=4&start=0')
INFO:li:scraper:('[Engineer][Europe]', 'Trying first selectors set')
INFO:li:scraper:('[Engineer][Europe]', 'Trying second selectors set')
INFO:li:scraper:('[Engineer][Europe]', 'Failed to load container selector, skip')


[ON_END]
